In [ ]:
library(tidyverse)
library(cowplot)
library(ggprism)
library(scales)
library(fs)
library(broom)
library(UpSetR)

## ANI Figure

In [ ]:
quality_results <- read_tsv("./data/checkm_sar86_quality_data_corrected.txt", show_col_types = FALSE)
# ANI subfigure figure
ani_values <- read_csv("./data/filtered_ani_values.csv", show_col_types = FALSE)
ani_figure <- ani_values %>%
  ggplot(aes(x = `ANI Value`)) + geom_histogram(binwidth = 1) + ylab("Genomes (Total No.)") + xlab("ANI (%)") + mike_formatting +
  scale_y_continuous(breaks = scales::pretty_breaks(n = 10))
alignment_frac_fig <- ani_values %>%
  ggplot(aes(x = `Alignment Fraction`)) + geom_histogram() + ylab("") + xlab("Alignment Fraction (%)") + mike_formatting +
  scale_y_continuous(breaks = scales::pretty_breaks(n = 10))
ani_frac_fig <- plot_grid(ani_figure, alignment_frac_fig, labels = "AUTO")

In [ ]:
# Load data on cluster sizes
ani_clusters_sizes <- read_csv("./data/ani_cluster_sizes.csv", show_col_types = FALSE)
# Load data on the size of each cluster within the 93% ANI clustering method
species_cluster_size <- read_csv("./data/species_cluster_sizes.csv", show_col_types = FALSE)
ani_clusters_sizes_fig <- ani_clusters_sizes %>%
  ggplot(aes(x = `ANI-value`, y = `Group-size`)) +
  geom_point() + 
  xlab("ANI (%)") +
  ylab("Species Clusters (Total No.)") +
  scale_y_continuous(guide = "prism_minor", limits = c(40, 165), expand = c(0, 0), minor_breaks = seq(0, 160, 5)) + 
  scale_x_continuous(guide = "prism_minor", limits = c(79, 100), expand = c(0, 0), minor_breaks = seq(80, 100, 1)) 

species_sizes_fig <- species_cluster_size %>%
  ggplot(aes(x = count)) +
  geom_histogram(binwidth = 1) +
  xlab("No. of Genomes in Cluster") +
  ylab("No. of Species Clusters") + 
  scale_x_continuous(guide = "prism_minor", minor_breaks = seq(0, 20, 1)) + 
  scale_y_continuous(guide = "prism_minor", limits = c(0, 55), expand = c(0, 0), minor_breaks = seq(0, 50, 2))

In [ ]:
ani_related_figure <-  plot_grid(ani_figure, alignment_frac_fig, ani_clusters_sizes_fig, species_sizes_fig, ncol = 2, nrow = 2, labels = c("a", "b", "c", "d"))
ani_related_figure

## Genome Qualities Figure

In [ ]:
# Load CheckM HIMB1674 corrected data
quality_data <- read_tsv("./data/checkm_sar86_quality_data_corrected.txt", show_col_types = FALSE)
# Load CheckM uncorrected data i.e. families 
quality_data_uncorrected <- read_tsv("./data/checkm_sar86_quality_data.txt", show_col_types = FALSE)
genome_data <- read_tsv("./data/storage/bin_stats.analyze.tsv", col_names = FALSE, show_col_types = FALSE)

In [ ]:
genome_data_cleaned <- genome_data %>%
  separate(col = X2, into = c("GC", "GC_std", "Genome_size", "#_ambiguous_bases", "#_scaffolds", "#_contigs",
                              "Longest_scaffold", "Longest_contig", "N50_(scaffolds)", "N50_(contigs)",
                              "Mean_scaffold_length", "Mean_contig_length", "Coding_density", "Translation_table",
                              "#_predicted_genes"), sep = ",") %>%
  mutate_all(list(~gsub(".*:", "", .))) %>%
  mutate_all(list(~gsub("}", "", .)))

In [ ]:
full_data <- quality_data %>%
  left_join(genome_data_cleaned, by = c(`Bin Id` = "X1"))

full_data_uncorrected <- quality_data_uncorrected %>%
  left_join(genome_data_cleaned, by = c(`Bin Id` = "X1"))

In [ ]:
species_clusters <- read_tsv("./data/sar86_species_clusters.txt", show_col_types = FALSE)
# Genome Taxonomy association file
taxonomy <- read_csv("./data/family_genus_viz.csv", show_col_types = FALSE)

# Drop version number for easier handling modification
taxonomy_rolling <- taxonomy %>%
    rename(genome_no_ext = Genome) %>%
    select(genome_no_ext, Family, Genus)

# The same everywhere else
full_data <- full_data %>%
  mutate(genome_no_version = gsub("\\.[1-2]", "", `Bin Id`))

full_data_uncorrected <- full_data_uncorrected %>%
  mutate(genome_no_version = gsub("\\.[1-2]", "", `Bin Id`))

In [ ]:
full_data <- taxonomy_rolling %>%
  left_join(full_data, by = c(genome_no_ext = "genome_no_version"))
full_data$GC <- as.numeric(full_data$GC)

full_data_uncorrected <- taxonomy_rolling %>%
  left_join(full_data_uncorrected, by = c(genome_no_ext = "genome_no_version"))
full_data_uncorrected$GC <- as.numeric(full_data_uncorrected$GC)

# Make sure that the length column is numeric
full_data$Genome_size <- as.numeric(full_data$Genome_size)
full_data_uncorrected$Genome_size <- as.numeric(full_data_uncorrected$Genome_size)

# We can also transform the length into Mbp instead of bp
full_data <- full_data %>%
  mutate(genome_size_mbp = Genome_size/1e+06)
full_data_uncorrected <- full_data_uncorrected %>%
  mutate(genome_size_mbp = Genome_size/1e+06)
# Ordering of families for visualizations and coloring
family_level_order <- c("Suzuki", "CHAB-I-7", "RedeBAC7D11", "SAR156")
sar86_colors <- c("#F17C6C", "#F8CB9A", "#E5EFBA", "#7AACA1")

In [ ]:
# Create scatterplot of completeness vs size of genome
qual_length_fig <- full_data_uncorrected %>%
    # Make it so that the shape for HIMB1674 is a sqaure so it sticks out
    mutate(shape_number = case_when(str_detect(`Bin Id`, "HIMB1674") ~ "Isolate", TRUE ~ "External")) %>%
    ggplot(aes(Completeness, genome_size_mbp, fill= Family)) +
        geom_point(aes(shape=shape_number), alpha = 0.9, size = 7, colour="black") +
        labs(y = "Genome Length (Mbp)", x = "Gammaproteobacteria-based Completeness (%)") +
        theme_grey(base_size = 22) +
        scale_fill_manual(values = c(`Suzuki` = sar86_colors[[1]], `CHAB-I-7` = sar86_colors[[2]], 
                                      RedeBAC7D11 = sar86_colors[[3]], SAR156 = sar86_colors[[4]])) +
        scale_shape_manual(values = c(21, 22)) +
        mike_formatting +
        scale_y_continuous(breaks = scales::pretty_breaks(n = 10)) +
        scale_x_continuous(limits = c(65, 100)) +
        guides(shape = "none")

In [ ]:
# GC content plot
family_gc_boxplot <- full_data %>%
    mutate(GC = GC*100) %>%
    {. ->> gc_data_stats } %>% # For stats analysis
    ggplot(aes(x = factor(Family, levels = c("Suzuki", "CHAB-I-7", "RedeBAC7D11", "SAR156")), y = GC, fill = Family)) + 
        geom_boxplot() + 
        theme_grey(base_size = 22) + labs(y = "GC Content (%)", x = "Family") +
        scale_fill_manual(breaks = family_level_order, values = sar86_colors) +
        xlab("") + 
        theme(legend.position = c(0.75, 0.85), legend.background = element_rect(fill = "white", color = "black"),
            axis.text.x=element_blank(), axis.ticks.x=element_blank()) +
        scale_y_continuous(breaks = scales::pretty_breaks(n = 10))

In [ ]:
# Initial stats test
kruskal.test(GC ~ Family, gc_data_stats)
# Post hoc
dunn_test(gc_data_stats, GC ~ Family, p.adjust.method = "bonferroni", detailed = FALSE)

In [ ]:
# Load data for genomes from each family with qualities estimated by their family specific marker gene sets
suzuki_focused_quals <- read_tsv("../data/30_family_qual_est/Suzuki/qual_data_family_corrected.txt",
                                 show_col_types = FALSE)
chab_focused_quals <- read_tsv("../data/30_family_qual_est/CHAB-I-7/qual_data_family_corrected.txt",
                               show_col_types = FALSE)
rede_focused_quals <- read_tsv("../data/30_family_qual_est/RedeBAC7D11/qual_data_family_corrected.txt",
                               show_col_types = FALSE)
sar156_focused_quals <- read_tsv("../data/30_family_qual_est/SAR156/qual_data_family_corrected.txt",
                                 show_col_types = FALSE)

In [ ]:
family_qual_estimates <- suzuki_focused_quals %>%
    rbind(chab_focused_quals) %>% 
    rbind(rede_focused_quals) %>%
    rbind(sar156_focused_quals) %>% 
    mutate(Genome = str_replace(`Bin Id`, "\\.[1-2]", "")) %>%
    relocate(Genome) %>%
    merge(taxonomy) %>% 
    mutate(est_size_mbp = (`Genome size (bp)` / (Completeness/100))/1000000) %>%
    mutate(size_mbp = (`Genome size (bp)` /1000000)) %>% relocate(size_mbp, .after = `Genome size (bp)`) %>%
    relocate(est_size_mbp, .after = size_mbp)

In [ ]:
# Create dataframe with himb1674 completion and size estimates
himb_size_est <- full_data %>%
    mutate(markerset = "HIMB1674-based") %>% 
    select(genome_no_ext, Family, estimatedsize, markerset) %>% 
    rename(est_size_mbp = estimatedsize, Genome = genome_no_ext)

In [ ]:
# Run stats to check differences between size estimates using HIMB1674 completion estimates
kruskal.test(est_size_mbp ~ Family, himb_size_est)
dunn_test(himb_size_est, est_size_mbp ~ Family, p.adjust.method = "bonferroni", detailed = FALSE)

In [ ]:
# Create dataframe with family-specific completion and size estimates
family_size_est <- family_qual_estimates %>% 
    mutate(markerset = "Family-based") %>% 
    select(Genome, Family, est_size_mbp, markerset) 

# Run stats to check differences between size estimates using family-sepcific completion estimates
kruskal.test(est_size_mbp ~ Family, family_size_est)
dunn_test(family_size_est, est_size_mbp ~ Family, p.adjust.method = "bonferroni", detailed = FALSE)

In [ ]:
# Combine the tables
himb_family_size_est <- family_size_est %>% rbind(himb_size_est)

In [ ]:
genome_size_boxplot_compare <- himb_family_size_est %>% 
    ggplot(aes(factor(Family, level = family_level_order), est_size_mbp, fill = interaction(markerset, Family))) + 
    geom_boxplot() +
    scale_y_continuous(breaks = scales::pretty_breaks(n = 10)) +
    theme(legend.position = c(0.25, 0.90), legend.background = element_rect(fill = "white", color = "black")) +
    xlab("") + 
    scale_fill_manual(values = c(sar86_colors[[2]], "purple", sar86_colors[[3]], "orange",
                                 sar86_colors[[4]], "green", sar86_colors[[1]], "blue")) +
    ylab("Predicted Genome Length (Mbp)")

In [ ]:
# Create initial subplots for the qualities related figure
three_plot <- plot_grid(qual_length_fig + theme(legend.position = "none"), (plot_grid(ncol = 1, family_gc_boxplot, genome_size_boxplot_compare)))

## Marker Gene Plot

In [ ]:
# Load data on marker genes present in each genome
marker_genes <- read_tsv("../data/02_sar86_quality/checkm_sar86_marker_data.txt", show_col_types = FALSE)
marker_genes_prop <- marker_genes %>% 
    rename(Genome = `Node Id: 2; Marker lineage: Gammaproteobacteria`) %>%
    mutate(Genome = str_replace(Genome, "\\.[1-2]", "")) %>% 
    merge(y = taxonomy) %>% 
    group_by(Family) %>% summarise(across(where(is.numeric), ~ sum(.x > 0)/n()))

In [ ]:
# Get marker genes present in more than 50% of genomes from each family
suzuki_marker_genes <- marker_genes_prop %>%
    filter(Family == "Suzuki") %>%
    select_if(~any(. >= 0.5)) %>% 
    select(-c("Family")) %>%
    colnames()
chab_marker_genes <- marker_genes_prop %>%
    filter(Family == "CHAB-I-7") %>%
    select_if(~any(. >= 0.5)) %>% 
    select(-c("Family")) %>%
    colnames()
rede_marker_genes <- marker_genes_prop %>%
    filter(Family == "RedeBAC7D11") %>%
    select_if(~any(. >= 0.5)) %>% 
    select(-c("Family")) %>%
    colnames()
sar156_marker_genes <- marker_genes_prop %>%
    filter(Family == "SAR156") %>%
    select_if(~any(. >= 0.5)) %>% 
    select(-c("Family")) %>%
    colnames()

In [ ]:
himb1674_genes <- marker_genes %>%
    filter(`Node Id: 2; Marker lineage: Gammaproteobacteria` == "HIMB1674") %>%
    select(-`Node Id: 2; Marker lineage: Gammaproteobacteria`) %>%
    select_if(~any(. >= 1)) %>%
    colnames()

In [ ]:
# Create the tibble of overlap in marker genes
# Create a tibble with the counts of entries in each vector
tibble_data <- tibble(
  Description = c("Suzuki", "CHAB-I-7", "RedeBAC7D11", "Magnimaribacteraceae"),
  Count = c(length(suzuki_marker_genes), length(chab_marker_genes), length(rede_marker_genes), length(sar156_marker_genes))
)

# Calculate overlaps
overlap_1_2 <- length(intersect(suzuki_marker_genes, chab_marker_genes))
overlap_3_4 <- length(intersect(rede_marker_genes, sar156_marker_genes))
overlap_all <- length(Reduce(intersect, list(suzuki_marker_genes, chab_marker_genes, rede_marker_genes, sar156_marker_genes)))

# Add rows with overlap information
tibble_data <- tibble_data %>%
    add_row(Description = "Suzuki & CHAB-I-7", Count = overlap_1_2) %>%
    add_row(Description = "RedeBAC7D11 & Magnimaribacteraceae", Count = overlap_3_4) %>%
    add_row(Description = "Magnimaribacterales", Count = overlap_all) %>%
    add_row(Description = "HIMB1674", Count = length(himb1674_genes))


# View the tibble
tibble_data <- tibble_data %>%
    mutate(Missing = 280-Count)


In [ ]:
tibble_data %>%
    pivot_longer(!Description, names_to = "Type", values_to = "Value") %>%
    # Rearrange so that Missing comes at the top
    mutate(Type = factor(Type, levels = c("Missing", "Count"))) %>%
    # Rearrange for custom x axis order
    mutate(Description = factor(Description, levels = c("HIMB1674", 
                                                        "Suzuki", 
                                                        "CHAB-I-7", 
                                                        "RedeBAC7D11", 
                                                        "Magnimaribacteraceae", 
                                                        "Suzuki & CHAB-I-7",
                                                        "RedeBAC7D11 & Magnimaribacteraceae",
                                                        "Magnimaribacterales"))) %>%
    # For custom fills
    mutate(Fill_interaction = interaction(Description, Type, sep = "-")) %>%
    mutate(transparency = ifelse(Type == "Missing", 0, 1)) %>%  # 1 for opaque, 0.5 for semi-transparent
    ggplot(aes(x = Description, y = Value, fill = Fill_interaction)) +
        geom_bar(stat = "identity", position = "stack", linewidth = 1, color = "black") +
        geom_text(aes(label = Value), position = position_stack(vjust = 0.95), color = "black", size = 12) +
        labs(x = "Taxonomic Grouping", y = "Marker genes (#)") +
        mike_formatting +
        scale_y_continuous(breaks = seq(0, 280, by = 50),
                           minor_breaks = seq(0, 280, by = 10),
                           guide=guide_axis(minor.ticks = TRUE)) +  # Set y-axis breaks
        theme(legend.position = "None",
              axis.text.x = element_text(margin = margin(t = 10)))

## Genus GC content figure

In [ ]:
sar86_generas_order <- c("Suzuki_G01", "Suzuki_G02", "Suzuki_G03", "Suzuki_G04", "Suzuki_G05", "Suzuki_G06", "Suzuki_G07", "Suzuki_G08", "Suzuki_G09", "Suzuki_G10", "CHAB-I-7_G1",
  "CHAB-I-7_G2", "CHAB-I-7_G3", "CHAB-I-7_G4", "RedeBAC7D11_G1", "RedeBAC7D11_G2", "RedeBAC7D11_G3", "RedeBAC7D11_G4", "SAR156_G1", "SAR156_G2", "SAR156_G3",
  "SAR156_G4")

genus_linkage_named_list <- setNames(sar86_generas_order, sar86_generas_order)
genus_linkage_named_list[["SAR156_G4"]] = "Magnimaribacter"
genera_gc_boxplot <- full_data %>%
    mutate(Genus = str_replace(Genus, ".*_", "")) %>%
    mutate(GC= GC*100) %>% 
    mutate(Family = factor(Family, levels = family_level_order)) %>%
    ggplot(aes(x = Genus, y = GC, fill = Family)) +
    geom_boxplot() +
    labs(y = "GC Content (%)", x = "Genera") +
    scale_fill_manual(breaks = family_level_order, labels=c("Suzuki", "CHAB-I-7", "RedeBAC7D11", "Magnimaribacteraceae"), values = sar86_colors) +
    mike_formatting +
    tilt_x_values +
    scale_y_continuous(breaks = scales::pretty_breaks(n = 10)) +
    scale_x_discrete(labels = genus_linkage_named_list) +
    ggh4x::facet_grid2(.~Family, scales="free", switch = "x") +
    ggh4x::force_panelsizes(cols = c(10, 4, 4, 4)) +
    theme(legend.position = c(0.90, 0.90),
            legend.background = element_rect(fill = "white", color = "black"),
            strip.background = element_blank(),
            strip.text.x = element_blank())